In [1]:


from pathlib import Path
from os.path import expanduser
from os import path
import numpy as np
import sys
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
import os


# In[2]:


root_folder = os.path.dirname(os.getcwd())
path_to_ground_truth_data = path.join(root_folder,'data/synthesized/gt_46.npy');
path_to_recording= path.join(root_folder,'data/synthesized/data_46.npy');

sys.path.append(root_folder)
from custom_resnet import CustomResnet as cnn


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

waveform_length = 72;
step_size = int(np.round(waveform_length * 0.1))
batch_size = 15000;


# In[3]:


# loads spike detection model
num_classes_detect = 2;
path_to_model_detect = path.join(root_folder, 'models/detect/resnet18_num_classes_2_epoch_22.pt')
model_detect = cnn.resnet18(num_classes=num_classes_detect);
checkpoint = torch.load(path_to_model_detect)
model_detect.load_state_dict(checkpoint['model_state_dict'])
model_detect.to(device);


# In[4]:


# loads re id model
num_classes_re_id = 499;
path_to_model_re_id = path.join(root_folder, 'models/re_id/resnet18_num_classes_499_epoch_74.pt')
model_re_id = cnn.ft_net(class_num = num_classes_re_id);
checkpoint = torch.load(path_to_model_re_id)
model_re_id.load_state_dict(checkpoint['model_state_dict'])
model_re_id.to(device);

acc_d = [];
fp_d = [];
total_d = [];
re_id_d = []
# In[5]:
for d in range(46, 95):
    path_to_ground_truth_data = path.join(root_folder,'data/synthesized/gt_'+ str(d)+'.npy');
    path_to_recording= path.join(root_folder,'data/synthesized/data_' + str(d) +'.npy');


    # transforms data for inference
    transform = transforms.Compose([cnn.FilterSignalUsingButtersWorth('high', 24000, np.array([100], dtype=int), 1), cnn.OptimizedZScoreNormalizaton()])
    recording = cnn.Recording(path_to_recording, transform = transform);
    recording = cnn.AddPaddingToRecording(recording, step_size, waveform_length)
    # gets waveform data for each window
    waveform_indices = cnn.GetWaveformIndices(recording, step_size, waveform_length)
    transform = transforms.Compose([cnn.ExtractWaveforms(waveform_indices, waveform_length)])
    waveforms = transform(recording.data);
    normalized_waveforms = waveforms;


    # In[6]:


    ### spike detection pipeline
    # loads data for inference
    dataset_to_infer_detect = cnn.InferenceDataset(normalized_waveforms.to(device));
    data_loader_detect = torch.utils.data.DataLoader(dataset_to_infer_detect, batch_size=batch_size, shuffle=False, num_workers=0);

    #inference for detection 
    result_detect = cnn.Inference(model_detect, data_loader_detect, num_classes_detect)
    treshold = 0.9;
    predictions = cnn.PredictionByTreshold(result_detect, treshold)
    max_amplitude_index = cnn.GetNonOverlappingSpikesMaxAmplitude(recording, predictions, waveform_length, waveforms, waveform_indices)


    # In[7]:


    del dataset_to_infer_detect, data_loader_detect
    torch.cuda.empty_cache()



    # In[8]:

    %pdb on
    non_out_of_bound_index = np.where((max_amplitude_index - waveform_length // 2 >= 0) & (max_amplitude_index + waveform_length // 2 < recording.__len__()));
    max_amplitude_index = max_amplitude_index[non_out_of_bound_index];

    max_amplitude_index = torch.unsqueeze(torch.tensor(max_amplitude_index), 0);
    transform = transforms.Compose([cnn.FilterSignalUsingButtersWorth('high', 24000, np.array([100], dtype=int), 1), cnn.OptimizedZScoreNormalizaton()])
    recording = cnn.Recording(path_to_recording, transform = transform);


    # In[9]:


    max_amplitude_index.shape


    # In[10]:


    # spike classficiation pipeline
    transform = transforms.Compose([cnn.ExtractWaveforms(max_amplitude_index, waveform_length)])
    waveforms_re_id = transform(recording.data);
    #vertical flip
    waveforms_re_id_flipped = waveforms_re_id * -1;
    # data loading for inference
    dataset_to_re_id = cnn.InferenceDataset(waveforms_re_id.to(device));
    data_loader_re_id = torch.utils.data.DataLoader(dataset_to_re_id, batch_size=batch_size, shuffle=False, num_workers=0);
    dataset_to_re_id_flipped = cnn.InferenceDataset(waveforms_re_id_flipped.to(device));
    data_loader_re_id_flipped = torch.utils.data.DataLoader(dataset_to_re_id_flipped, batch_size=batch_size, shuffle=False, num_workers=0);


    # In[ ]:





    # In[11]:


    # exracts features
    feature_map_dims = 128
    features = torch.FloatTensor(0 , 128).zero_();
    model_re_id.model.fc.classifier = nn.Sequential()
    result_re_id = cnn.Inference(model_re_id, data_loader_re_id, feature_map_dims)
    result_re_id_flipped = cnn.Inference(model_re_id, data_loader_re_id_flipped, feature_map_dims)
    features = result_re_id + result_re_id_flipped ;
    temp = torch.norm(features, p=2, dim=1, keepdim=True)
    unit_features = features.div(temp.expand_as(features))


    # In[12]:


    del dataset_to_re_id, data_loader_re_id, dataset_to_re_id_flipped, data_loader_re_id_flipped
    torch.cuda.empty_cache()



    # In[13]:


    treshold = 0.6;
    feature_list = torch.unsqueeze(unit_features[0, :], 0).to(device)
    ids_list = torch.tensor([1], dtype=torch.int)
    for i in range(1,unit_features.size()[0]):
        query = torch.unsqueeze(unit_features[i, :], 0).to(device);
        query_raveled = query.view(-1,1);
        score = torch.mm(feature_list, query_raveled);
        max_val, max_ind = torch.max(score, 0);
        # spike is already in list
        if(max_val.item() > treshold):
            ids_list = torch.cat((ids_list, torch.tensor([ids_list[max_ind.item()]], dtype = torch.int)), 0);
        # adds a new spike id
        else:
            new_id = torch.max(ids_list, 0)[0] + 1;
            ids_list = torch.cat((ids_list, torch.tensor([new_id.item()], dtype = torch.int)), 0);

        feature_list = torch.cat((feature_list, query), 0);


    # In[14]:


    ids_list[0:100]


    # In[ ]:





    # In[15]:


    temp =np.load(path_to_ground_truth_data)


    # In[ ]:





    # In[16]:


    unique_classes, nb_of_occourences = np.unique(ids_list, return_counts=True);
    print(unique_classes)
    print(nb_of_occourences)


    # In[17]:


    unique_classes_gt, nb_of_occourences_gt = np.unique(temp[1, :], return_counts=True);
    print(unique_classes_gt)
    print(nb_of_occourences_gt)


    # In[18]:


    unique_classes_gt, nb_of_occourences_gt = np.unique(ids_list, return_counts=True);
    print(unique_classes_gt)
    print(nb_of_occourences_gt)


    # In[19]:


    found_spike_index = [];
    found = [];
    for i in range(max_amplitude_index.size()[1]):
        spike = max_amplitude_index[0, i];
        ind =np.where(spike.numpy() == temp[0, :].astype(int))
        if(len(ind[0]) == 0):
            found.append(0);
            found_spike_index.append(0)

        else:
            found.append(1)
            found_spike_index.append(int(temp[1, ind[0]]) + 1)


    # In[20]:


    found = np.asarray(found)
    found_spike_index = np.asarray(found_spike_index)


    # In[21]:


    tp = np.where(found == 1)[0].size
    fp = np.where(found == 0)[0].size
    print(tp / temp[1, :].size)
    print(fp)
    acc_d.append(tp / temp[1, :].size);
    fp_d.append(fp);
    total_d.append(temp[1, :].size);

    # In[22]:


    found.size


    # In[23]:


    unique_classes, nb_of_occourences = np.unique(found_spike_index, return_counts=True);
    print(unique_classes)
    print(nb_of_occourences)


    # In[24]:


    unique_classes_gt, nb_of_occourences_gt = np.unique(temp[1, :], return_counts=True);
    print(unique_classes_gt + 1)
    print(nb_of_occourences_gt)


    # In[ ]:





    # In[ ]:





    # In[25]:


    unique_classes, nb_of_occourences = np.unique(found_spike_index, return_counts=True);

    spike_classes = np.empty((unique_classes.size,),dtype=object)
    for i in range(spike_classes.size):
        spike_classes[i] = [];
    for i in range(found_spike_index.size):
        index = found_spike_index[i];
        index_re_id = ids_list[i]
        if (index == 0):
            spike_classes[index].append(index_re_id.item());
        else:
            spike_classes[index].append(index_re_id.item())




    # In[ ]:





    # In[26]:


    tp = 0;
    total = 0;
    for i in range(1, spike_classes.size):
        unique_classes, nb_of_occourences = np.unique(spike_classes[i], return_counts=True);
        tp = tp + np.max(nb_of_occourences);
        total = total + np.sum(nb_of_occourences);
        print("max occ: ", np.max(nb_of_occourences), "total: ", np.sum(nb_of_occourences), "id: ", unique_classes[np.argmax(nb_of_occourences)])



    # In[27]:


    tp / total
    re_id_d.append(tp / total)


    # In[28]:


    unique_classes, nb_of_occourences = np.unique(spike_classes[0], return_counts=True);
    print(nb_of_occourences)


    # In[ ]:





[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
torch.cuda.FloatTensor
done:  0  batch
done:  1  batch
done:  2  batch
done:  3  batch
done:  4  batch
done:  5  batch
done:  6  batch
done:  7  batch
done:  8  batch
done:  9  batch
done:  10  batch
done:  11  batch
done:  12  batch
done:  13  batch
done:  14  batch
done:  15  batch
done:  16  batch
done:  17  batch
done:  18  batch
done:  19  batch
done:  20  batch
done:  21  batch
done:  22  batch
done:  23  batch
done:  24  batch
done:  25  batch
done:  26  batch
done:  27  batch
done:  28  batch
done:  29  batch
done:  30  batch
done:  31  batch
done:  32  batch
done:  33  batch
done:  34  batch
done:  35  batch
done:  36  batch
done:  37  batch
done:  38  batch
done:  39  batch
done:  40  batch
done:  41  batch
done:  42  batch
done:  43  batch
done:  44  batch
done:  45  batch
done:  46  batch
done:  47  batch
done:  48  batch
done:  49  batch
done:  50  batch
done:  51  batch
done:  52  batch
done:  53  batch
done:  54  batch


done:  56  batch
done:  57  batch
done:  58  batch
done:  59  batch
done:  60  batch
done:  61  batch
done:  62  batch
done:  63  batch
done:  64  batch
done:  65  batch
done:  66  batch
done:  67  batch
done:  68  batch
done:  69  batch
done:  70  batch
done:  71  batch
done:  72  batch
done:  73  batch
done:  74  batch
done:  75  batch
done:  76  batch
done:  77  batch
done:  78  batch
done:  79  batch
done:  80  batch
done:  81  batch
done:  82  batch
done:  83  batch
done:  84  batch
done:  85  batch
done:  86  batch
done:  87  batch
done:  88  batch
done:  89  batch
done:  90  batch
done:  91  batch
done:  92  batch
done:  93  batch
done:  94  batch
done:  95  batch
done:  96  batch
done:  97  batch
done:  98  batch
done:  99  batch
done:  100  batch
done:  101  batch
done:  102  batch
done:  103  batch
done:  104  batch
done:  105  batch
done:  106  batch
done:  107  batch
done:  108  batch
done:  109  batch
done:  110  batch
done:  111  batch
done:  112  batch
done:  113  batch


done:  108  batch
done:  109  batch
done:  110  batch
done:  111  batch
done:  112  batch
done:  113  batch
done:  114  batch
done:  115  batch
done:  116  batch
done:  117  batch
done:  118  batch
done:  119  batch
done:  120  batch
done:  121  batch
done:  122  batch
done:  123  batch
done:  124  batch
done:  125  batch
done:  126  batch
done:  127  batch
done:  128  batch
done:  129  batch
done:  130  batch
done:  131  batch
done:  132  batch
done:  133  batch
done:  134  batch
done:  135  batch
done:  136  batch
done:  137  batch
Automatic pdb calling has been turned ON
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
torch.cuda.FloatTensor
torch.cuda.FloatTensor
done:  0  batch
done:  0  batch
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32]
[1283 2318    4  784  958  704    7   60   76  835  737  350  670   35
  487    1   14  732    3 1052  564    1   22    7  170  112    1    1
    1    1    1    1]
[ 0.  1.  2.  3.  4.  5.  

done:  84  batch
done:  85  batch
done:  86  batch
done:  87  batch
done:  88  batch
done:  89  batch
done:  90  batch
done:  91  batch
done:  92  batch
done:  93  batch
done:  94  batch
done:  95  batch
done:  96  batch
done:  97  batch
done:  98  batch
done:  99  batch
done:  100  batch
done:  101  batch
done:  102  batch
done:  103  batch
done:  104  batch
done:  105  batch
done:  106  batch
done:  107  batch
done:  108  batch
done:  109  batch
done:  110  batch
done:  111  batch
done:  112  batch
done:  113  batch
done:  114  batch
done:  115  batch
done:  116  batch
done:  117  batch
done:  118  batch
done:  119  batch
done:  120  batch
done:  121  batch
done:  122  batch
done:  123  batch
done:  124  batch
done:  125  batch
done:  126  batch
done:  127  batch
done:  128  batch
done:  129  batch
done:  130  batch
done:  131  batch
done:  132  batch
done:  133  batch
done:  134  batch
done:  135  batch
done:  136  batch
done:  137  batch
Automatic pdb calling has been turned ON
[ 0

torch.cuda.FloatTensor
torch.cuda.FloatTensor
done:  0  batch
done:  0  batch
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14]
[3892  373  720  806  483  960  749    2    2    8    2    1    2    2]
[0. 1. 2. 3. 4. 5.]
[2799  373  765 1214  810  960]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14]
[3892  373  720  806  483  960  749    2    2    8    2    1    2    2]
0.9543418581129894
1397
[0 1 2 3 4 5 6]
[1397 2578  373  763 1181  774  936]
[1. 2. 3. 4. 5. 6.]
[2799  373  765 1214  810  960]
max occ:  2575 total:  2578 id:  1
max occ:  373 total:  373 id:  2
max occ:  745 total:  763 id:  7
max occ:  702 total:  1181 id:  3
max occ:  772 total:  774 id:  4
max occ:  936 total:  936 id:  6
[1303   17   34   15   24    2    1    1]
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
torch.cuda.FloatTensor
done:  0  batch
done:  1  batch
done:  2  batch
done:  3  batch
done:  4  batch
done:  5  batch
done:  6  batch
done:  7  batch
done:  8  batch
done:  9  batch
done:  10  batch
done:  11  bat

...

[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
torch.cuda.FloatTensor
done:  0  batch
done:  1  batch
done:  2  batch
done:  3  batch
done:  4  batch
done:  5  batch
done:  6  batch
done:  7  batch
done:  8  batch
done:  9  batch
done:  10  batch
done:  11  batch
done:  12  batch
done:  13  batch
done:  14  batch
done:  15  batch
done:  16  batch
done:  17  batch
done:  18  batch
done:  19  batch
done:  20  batch
done:  21  batch
done:  22  batch
done:  23  batch
done:  24  batch
done:  25  batch
done:  26  batch
done:  27  batch
done:  28  batch
done:  29  batch
done:  30  batch
done:  31  batch
done:  32  batch
done:  33  batch
done:  34  batch
done:  35  batch
done:  36  batch
done:  37  batch
done:  38  batch
done:  39  batch
done:  40  batch
done:  41  batch
done:  42  batch
done:  43  batch
done:  44  batch
done:  45  batch
done:  46  batch
done:  47  batch
done:  48  batch
done:  49  batch
done:  50  batch
done:  51  batch
done:  52  batch
done:  53  batch
done:  54  batch


torch.cuda.FloatTensor
done:  0  batch
done:  1  batch
done:  2  batch
done:  3  batch
done:  4  batch
done:  5  batch
done:  6  batch
done:  7  batch
done:  8  batch
done:  9  batch
done:  10  batch
done:  11  batch
done:  12  batch
done:  13  batch
done:  14  batch
done:  15  batch
done:  16  batch
done:  17  batch
done:  18  batch
done:  19  batch
done:  20  batch
done:  21  batch
done:  22  batch
done:  23  batch
done:  24  batch
done:  25  batch
done:  26  batch
done:  27  batch
done:  28  batch
done:  29  batch
done:  30  batch
done:  31  batch
done:  32  batch
done:  33  batch
done:  34  batch
done:  35  batch
done:  36  batch
done:  37  batch
done:  38  batch
done:  39  batch
done:  40  batch
done:  41  batch
done:  42  batch
done:  43  batch
done:  44  batch
done:  45  batch
done:  46  batch
done:  47  batch
done:  48  batch
done:  49  batch
done:  50  batch
done:  51  batch
done:  52  batch
done:  53  batch
done:  54  batch
done:  55  batch
done:  56  batch
done:  57  batch
d

done:  82  batch
done:  83  batch
done:  84  batch
done:  85  batch
done:  86  batch
done:  87  batch
done:  88  batch
done:  89  batch
done:  90  batch
done:  91  batch
done:  92  batch
done:  93  batch
done:  94  batch
done:  95  batch
done:  96  batch
done:  97  batch
done:  98  batch
done:  99  batch
done:  100  batch
done:  101  batch
done:  102  batch
done:  103  batch
done:  104  batch
done:  105  batch
done:  106  batch
done:  107  batch
done:  108  batch
done:  109  batch
done:  110  batch
done:  111  batch
done:  112  batch
done:  113  batch
done:  114  batch
done:  115  batch
done:  116  batch
done:  117  batch
done:  118  batch
done:  119  batch
done:  120  batch
done:  121  batch
done:  122  batch
done:  123  batch
done:  124  batch
done:  125  batch
done:  126  batch
done:  127  batch
done:  128  batch
done:  129  batch
done:  130  batch
done:  131  batch
done:  132  batch
done:  133  batch
done:  134  batch
done:  135  batch
done:  136  batch
done:  137  batch
Automatic 

done:  133  batch
done:  134  batch
done:  135  batch
done:  136  batch
done:  137  batch
Automatic pdb calling has been turned ON
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
torch.cuda.FloatTensor
torch.cuda.FloatTensor
done:  0  batch
done:  0  batch
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39]
[ 590  390  140 1006 1060   55  623 3602   10  216  982  546  715   51
  619    2    1  326    9  254    6  329    1    1   61   10    1   67
    5   70   10    4    1    1    1    1    2    1    1]
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16.]
[2787  391   72  254   67  215  595 1005 1037 1060  619  326  106  623
  777  336  547]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39]
[ 590  390  140 1006 1060   55  623 3602   10  216  982  546  715   51
  619    2    1  326    9  254    6  329    1    1   61   10    1   67


done:  131  batch
done:  132  batch
done:  133  batch
done:  134  batch
done:  135  batch
done:  136  batch
done:  137  batch
Automatic pdb calling has been turned ON
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
torch.cuda.FloatTensor
torch.cuda.FloatTensor
done:  0  batch
done:  0  batch
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
[4034  388  209    1    8  890  196    4  439   68    5   72  885    1
    1    1    2    1    1    1]
[0. 1. 2. 3. 4. 5. 6. 7.]
[2939  196  314  439  858  895   72  398]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
[4034  388  209    1    8  890  196    4  439   68    5   72  885    1
    1    1    2    1    1    1]
0.9531991490754378
1382
[0 1 2 3 4 5 6 7 8]
[1382 2695  196  314  437  851  866   72  394]
[1. 2. 3. 4. 5. 6. 7. 8.]
[2939  196  314  439  858  895   72  398]
max occ:  2684 total:  2695 id:  1
max occ:  196 total:  196 id:  7
max occ:  184 total:  314 id:  3
max occ:  437 total:  437 id:  9
max occ:  81

In [3]:
s = 0;
for i in range(len(fp_d)):
    s = s +  fp_d[i] / total_d[i];
print("fp: ",s / len(fp_d))
s = 0;
for i in range(len(re_id_d)):
    s = s +  re_id_d[i];
print("reid: ",s / len(re_id_d))
s = 0;
for i in range(len(acc_d)):
    s = s +  acc_d[i];
print("acc_d: ",s / len(acc_d))

fp:  0.18167414941231305
reid:  0.943522406161278
acc_d:  0.9498523814512679
